In [1]:
import cx_Oracle
# 라이브러리 임포트
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Text, Date
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2
from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

In [37]:
# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')
# 커서 생성
cursor = connection.cursor()

In [5]:
#db에서 가져옴
query = "select * from TEAM"
Teamdf = pd.read_sql(query, con=engine)

In [6]:
Teamdf

,team_code,team_name,team_img_url
0,HH,한화,None
1,HT,KIA,None
2,KT,KT,None
3,LT,롯데,None
4,OB,두산,None
5,SS,삼성,None
6,SK,SSG,None
7,LG,LG,None
8,WO,키움,None
9,NC,NC,None


In [7]:
#db에서 가져옴
query = "select * from MATCHWEATHER"
Matchwdf = pd.read_sql(query, con=engine)

In [8]:
Matchwdf

,날짜,위치,최저기온,최고기온,강우확률
0,2024-09-05,창원,22°,33°,0%
1,2024-09-06,창원,25°,32°,60%
2,2024-09-07,창원,25°,31°,60%
3,2024-09-08,창원,24°,32°,30%
4,2024-09-09,창원,24°,32°,30%
...,...,...,...,...,...
85,2024-09-10,수원,23°,32°,20%
86,2024-09-11,수원,23°,32°,30%
87,2024-09-12,수원,24°,30°,40%
88,2024-09-13,수원,21°,29°,40%


In [15]:
#db에서 가져옴
query = "select * from MATCH_SCHEDULE order by match_date asc"
Matchdf = pd.read_sql(query, con=engine)

In [16]:
Matchdf

,match_id,match_date,match_time,status,team1_code,team2_code,home_team_code
0,1,2024-09-03,18:30,예정,LG,HT,HT
1,2,2024-09-03,18:30,예정,LT,SS,SS
2,3,2024-09-03,18:30,예정,OB,HH,HH
3,4,2024-09-03,18:30,예정,WO,NC,NC
4,5,2024-09-04,18:30,예정,HH,HT,HT
...,...,...,...,...,...,...,...
90,91,2024-09-28,17:00,예정,HT,LT,LT
91,92,2024-09-28,17:00,예정,LG,SS,SS
92,93,2024-09-28,17:00,예정,OB,NC,NC
93,94,2024-09-28,17:00,예정,SK,HH,HH


In [17]:
#db에서 가져옴
query = "select * from stadium_locations"
stadiumdf = pd.read_sql(query, con=engine)

In [18]:
stadiumdf

,team_code,team_name,stadium_name,std_image_url,latitude,longitude
0,SS,삼성,삼성 라이온즈 파크,https://i.namu.wiki/i/Y-V9qfbukjWKQN_sHjaj0lNp...,35.8400,128.6800
1,OB,두산,잠실 야구장,https://i.namu.wiki/i/OKi3ZiXPcaSCs5jj9Rx6KrAd...,37.5121,127.0710
2,LT,롯데,사직 야구장,https://i.namu.wiki/i/KV7IyWApI2PG22H5h51rI0Si...,35.1945,129.0630
3,LG,LG,잠실 야구장,https://i.namu.wiki/i/OKi3ZiXPcaSCs5jj9Rx6KrAd...,37.5121,127.0710
4,HT,KIA,광주-KIA 챔피언스 필드,https://i.namu.wiki/i/fC5S84T3kO_AMvFqKGpPsZZo...,35.1847,126.8520
5,HH,한화,대전 한화생명이글스파크,https://i.namu.wiki/i/eLXWwgX2zfxeCZDOGsoJnsBA...,36.3173,127.4200
6,NC,NC,창원 NC파크,https://i.namu.wiki/i/v8pBY2eV08-UW0ZgRbzXf0AG...,35.1807,128.5820
7,SK,SSG,인천 SSG 랜더스필드,https://www.ssglanders.com/img/stadium/img_bal...,37.4350,126.6930
8,WO,키움,고척 스카이돔,https://i.namu.wiki/i/iyvLMbwVmm-GR20XzJZHr4TL...,37.4982,126.8679
9,KT,KT,수원 KT위즈파크,https://i.namu.wiki/i/KXDRI0xADcyE0_V8mYcpINiM...,37.2999,127.0090


In [30]:
# 팀 코드에 따른 location 매핑
location_mapping = {
    'SS': '대구',
    'OB': '잠실',
    'LT': '사직',
    'HT': '광주',
    'HH': '대전',
    'NC': '창원',
    'SK': '인천',
    'WO': '고척',
    'KT': '수원',
    'LG': '잠실'
}

In [31]:
#location 컬럼추가
# stadiumdf에 location 컬럼 추가 및 매핑된 값 할당
Matchdf['location'] = Matchdf['home_team_code'].map(location_mapping)

In [32]:
Matchdf

,match_id,match_date,match_time,status,team1_code,team2_code,home_team_code,location
0,1,2024-09-03,18:30,예정,LG,HT,HT,광주
1,2,2024-09-03,18:30,예정,LT,SS,SS,대구
2,3,2024-09-03,18:30,예정,OB,HH,HH,대전
3,4,2024-09-03,18:30,예정,WO,NC,NC,창원
4,5,2024-09-04,18:30,예정,HH,HT,HT,광주
...,...,...,...,...,...,...,...,...
90,91,2024-09-28,17:00,예정,HT,LT,LT,사직
91,92,2024-09-28,17:00,예정,LG,SS,SS,대구
92,93,2024-09-28,17:00,예정,OB,NC,NC,창원
93,94,2024-09-28,17:00,예정,SK,HH,HH,대전


In [35]:
# 컬럼 이름을 병합 기준에 맞게 변경
Matchwdf.rename(columns={'날짜': 'match_date', '위치': 'location'}, inplace=True)

# 날짜 형식 변환 (datetime 형식으로 변환하여 일치하도록 설정)
Matchdf['match_date'] = pd.to_datetime(Matchdf['match_date'])
Matchwdf['match_date'] = pd.to_datetime(Matchwdf['match_date'])

# 두 데이터프레임을 날짜와 위치 기준으로 병합
merged_df = pd.merge(Matchdf, Matchwdf, on=['match_date', 'location'], how='inner')

# 결과 출력
print(merged_df)

    match_id match_date match_time status team1_code team2_code  \
0         10 2024-09-05      18:30     예정         HH         HT   
1         11 2024-09-05      18:30     예정         KT         LT   
2         12 2024-09-05      18:30     예정         SK         LG   
3         13 2024-09-05      18:30     예정         WO         NC   
4         14 2024-09-06      18:30     예정         HH         LG   
5         15 2024-09-06      18:30     예정         KT         NC   
6         16 2024-09-06      18:30     예정         SS         LT   
7         17 2024-09-06      18:30     예정         WO         HT   
8         18 2024-09-07      14:00     예정         HH         LG   
9         19 2024-09-07      17:00     예정         NC         SS   
10        20 2024-09-07      17:00     예정         OB         KT   
11        21 2024-09-07      17:00     예정         SK         LT   
12        22 2024-09-07      17:00     예정         WO         HT   
13        23 2024-09-08      14:00     예정         HH         L

In [36]:
merged_df

,match_id,match_date,match_time,status,team1_code,team2_code,home_team_code,location,최저기온,최고기온,강우확률
0,10,2024-09-05,18:30,예정,HH,HT,HT,광주,22°,32°,20%
1,11,2024-09-05,18:30,예정,KT,LT,LT,사직,23°,31°,0%
2,12,2024-09-05,18:30,예정,SK,LG,LG,잠실,24°,29°,60%
3,13,2024-09-05,18:30,예정,WO,NC,NC,창원,22°,33°,0%
4,14,2024-09-06,18:30,예정,HH,LG,LG,잠실,24°,31°,60%
5,15,2024-09-06,18:30,예정,KT,NC,NC,창원,25°,32°,60%
6,16,2024-09-06,18:30,예정,SS,LT,LT,사직,25°,31°,60%
7,17,2024-09-06,18:30,예정,WO,HT,HT,광주,24°,31°,60%
8,18,2024-09-07,14:00,예정,HH,LG,LG,잠실,23°,31°,20%
9,19,2024-09-07,17:00,예정,NC,SS,SS,대구,23°,31°,60%


In [45]:
merged_df.to_sql(
    'match_weather', 
    con=engine, 
    if_exists='append',  # 기존 테이블에 데이터 추가
    index=False, 
    dtype={
        'match_id': NUMBER,
        'match_date': Date(),
        'match_time': VARCHAR2(20),
        'status': VARCHAR2(20),
        'team1_code': VARCHAR2(10),
        'team2_code': VARCHAR2(10),
        'home_team_code': VARCHAR2(10),
        'location': VARCHAR2(50),
        '최저기온': VARCHAR2(10),
        '최고기온': VARCHAR2(10),
        '강우확률': VARCHAR2(10)
    }
)
print("Table insert successfully")

Table insert successfully
